In [1]:
%run common.ipynb

['LC_Type1', 'LST_Day_1km', 'pr', 'aet', 'NDVI', 'EVI']
['LST_Day_1km', 'pr', 'aet', 'NDVI', 'EVI']
['LC_Type1']


In [2]:
%run tuning_common.ipynb

Using TensorFlow backend


In [3]:
MODEL_SAVE_DIR = f'{DOWNSTREAM_E2E_DIR}model'

In [4]:
downstream_train_df = pd.read_csv(f'{DATA_FOLDER}{DOWNSTREAM_TRAIN_FILENAME}')
downstream_test_df = pd.read_csv(f'{DATA_FOLDER}{DOWNSTREAM_TEST_FILENAME}')

In [5]:
downstream_train_df = embed_catgorical_features(downstream_train_df, CATEGORICAL_FEATURES)
downstream_test_df = embed_catgorical_features(downstream_test_df, CATEGORICAL_FEATURES)

In [6]:
print(downstream_test_df.dtypes)
print(CATEGORICAL_FEATURES)

LC_Type1        object
LST_Day_1km    float64
pr             float64
aet            float64
NDVI           float64
EVI            float64
Gpp            float64
dtype: object
['LC_Type1']


In [7]:
from sklearn.model_selection import train_test_split
split_train_df, split_val_df = train_test_split(downstream_train_df, test_size=0.2, shuffle=True, random_state=8)

In [8]:
split_train_tf = df_to_dataset(split_train_df[FEATURES + [TARGET_FEATURE]], TARGET_FEATURE, shuffle=True, batch_size=256)
split_val_tf = df_to_dataset(split_val_df[FEATURES + [TARGET_FEATURE]], TARGET_FEATURE, shuffle=False, batch_size=256)

/home/zheng.yant/ML-LUE/fttransformer/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/home/zheng.yant/ML-LUE/fttransformer/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]


In [9]:
pretrained_model = tf.keras.models.load_model(f'{UPSTREAM_FINETUNE_OPTUNA_DIR}/model_backup')

## End to end tuning

In [10]:
input_tensors = get_input_tensors(split_train_tf)
outputs = pretrained_model(input_tensors, training=False)
model = tf.keras.Model(input_tensors, outputs)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 EVI (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 LC_Type1 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 LST_Day_1km (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 NDVI (InputLayer)              [(None, 1)]          0           []                               
                                                                                              

In [11]:
new_data_metrics = 'val_ft_transformer_1_root_mean_squared_error'
mode = 'min'
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor=new_data_metrics, mode=mode, patience=50, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(
        monitor=new_data_metrics,
        mode=mode,
        filepath=MODEL_SAVE_DIR,
        save_best_only=True,
        save_freq='epoch')
]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss = tf.keras.losses.MeanSquaredError(),
    metrics= [tf.keras.metrics.RootMeanSquaredError()]
)
tf.keras.backend.clear_session()
history = model.fit(
    split_train_tf, 
    epochs=1000, 
    validation_data=split_val_tf,
    callbacks=[callbacks]
)

Epoch 1/1000
4/4 [==============================] - ETA: 0s - loss: 164095056.0000 - ft_transformer_loss: 145342032.0000 - ft_transformer_1_loss: 18753038.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 4330.4780

4/4 [==============================] - 12s 2s/step - loss: 164095056.0000 - ft_transformer_loss: 145342032.0000 - ft_transformer_1_loss: 18753038.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 4330.4780 - val_loss: 169947104.0000 - val_ft_transformer_loss: 148593408.0000 - val_ft_transformer_1_loss: 21353704.0000 - val_ft_transformer_root_mean_squared_error: 12189.8906 - val_ft_transformer_1_root_mean_squared_error: 4621.0068
Epoch 2/1000
4/4 [==============================] - ETA: 0s - loss: 162534688.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 17192670.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 4146.4043

4/4 [==============================] - 7s 2s/step - loss: 162534688.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 17192670.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 4146.4043 - val_loss: 168841600.0000 - val_ft_transformer_loss: 148593424.0000 - val_ft_transformer_1_loss: 20248180.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4499.7979
Epoch 3/1000
4/4 [==============================] - ETA: 0s - loss: 161723024.0000 - ft_transformer_loss: 145342000.0000 - ft_transformer_1_loss: 16381017.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 4047.3469

4/4 [==============================] - 7s 2s/step - loss: 161723024.0000 - ft_transformer_loss: 145342000.0000 - ft_transformer_1_loss: 16381017.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 4047.3469 - val_loss: 168088000.0000 - val_ft_transformer_loss: 148593392.0000 - val_ft_transformer_1_loss: 19494610.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4415.2700
Epoch 4/1000
4/4 [==============================] - ETA: 0s - loss: 160803088.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 15461078.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3932.0576

4/4 [==============================] - 7s 2s/step - loss: 160803088.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 15461078.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3932.0576 - val_loss: 167670272.0000 - val_ft_transformer_loss: 148593408.0000 - val_ft_transformer_1_loss: 19076870.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4367.7075
Epoch 5/1000
4/4 [==============================] - ETA: 0s - loss: 160275744.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 14933720.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3864.4172

...

4/4 [==============================] - 7s 2s/step - loss: 156731952.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 11389942.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3374.8989 - val_loss: 165500192.0000 - val_ft_transformer_loss: 148593424.0000 - val_ft_transformer_1_loss: 16906764.0000 - val_ft_transformer_root_mean_squared_error: 12189.8906 - val_ft_transformer_1_root_mean_squared_error: 4111.7837
Epoch 70/1000
4/4 [==============================] - ETA: 0s - loss: 157680304.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 12338293.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3512.5906

4/4 [==============================] - 7s 2s/step - loss: 157680304.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 12338293.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3512.5906 - val_loss: 165325648.0000 - val_ft_transformer_loss: 148593392.0000 - val_ft_transformer_1_loss: 16732257.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4090.5081
Epoch 71/1000
4/4 [==============================] - 4s 839ms/step - loss: 157393680.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 12051655.0000 - ft_transformer_root_mean_squared_error: 12055.7871 - ft_transformer_1_root_mean_squared_error: 3471.5493 - val_loss: 166241968.0000 - val_ft_transformer_loss: 148593392.0000 - val_ft_transformer_1_loss: 17648578.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4201.0210
Epoch 72/1000
4

4/4 [==============================] - 7s 2s/step - loss: 156664176.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 11322145.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3364.8396 - val_loss: 165287376.0000 - val_ft_transformer_loss: 148593392.0000 - val_ft_transformer_1_loss: 16693983.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4085.8271
Epoch 74/1000
4/4 [==============================] - 3s 785ms/step - loss: 156198592.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 10856587.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3294.9336 - val_loss: 165815376.0000 - val_ft_transformer_loss: 148593408.0000 - val_ft_transformer_1_loss: 17221962.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4149.9351
Epoch 75/1000
4

4/4 [==============================] - 7s 2s/step - loss: 156982672.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 11640667.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3411.8423 - val_loss: 165270320.0000 - val_ft_transformer_loss: 148593408.0000 - val_ft_transformer_1_loss: 16676908.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4083.7371
Epoch 82/1000
4/4 [==============================] - 3s 719ms/step - loss: 156420736.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 11078705.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3328.4688 - val_loss: 165727200.0000 - val_ft_transformer_loss: 148593392.0000 - val_ft_transformer_1_loss: 17133814.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4139.3013
Epoch 83/1000
4

4/4 [==============================] - 7s 2s/step - loss: 155541056.0000 - ft_transformer_loss: 145342016.0000 - ft_transformer_1_loss: 10199045.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3193.5945 - val_loss: 165267648.0000 - val_ft_transformer_loss: 148593424.0000 - val_ft_transformer_1_loss: 16674230.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4083.4092
Epoch 110/1000
4/4 [==============================] - 4s 839ms/step - loss: 155310320.0000 - ft_transformer_loss: 145342032.0000 - ft_transformer_1_loss: 9968293.0000 - ft_transformer_root_mean_squared_error: 12055.7881 - ft_transformer_1_root_mean_squared_error: 3157.2603 - val_loss: 165270144.0000 - val_ft_transformer_loss: 148593392.0000 - val_ft_transformer_1_loss: 16676749.0000 - val_ft_transformer_root_mean_squared_error: 12189.8896 - val_ft_transformer_1_root_mean_squared_error: 4083.7175
Epoch 111/1000


In [12]:
plot_model_history(history, 'ft_transformer_1_root_mean_squared_error', new_data_metrics,'RMSE')

In [16]:
model_reload = tf.keras.models.load_model(MODEL_SAVE_DIR)

In [17]:
test_df = df_to_dataset(downstream_test_df[FEATURES], shuffle=False, batch_size=1)
predict_results = model_reload.predict(test_df)
y_pred = predict_results['output'].ravel()
modis_compare_data = get_modis_compare_data(y_pred)
get_all_metrics(y_pred, modis_compare_data[TARGET_FEATURE], downstream_test_df[TARGET_FEATURE])

/home/zheng.yant/ML-LUE/fttransformer/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]


156/156 [==============================] - 2s 9ms/step
RMSE prediction: 3760.3045
RMSE MOD17: 4553.6565
R-Squared prediction: 0.660331134625151
R-Squared MOD17: 0.5018842973195095
Max error prediction: 11849.962453125001
Max error MOD17: 15912.398000000001
Mean poisson deviance prediction: 1211.8268374592437
Mean poisson deviance MOD17: 1798.8400467873778
Mean gamma deviance prediction: 0.14518373642523288
Mean gamma deviance MOD17: 0.2386320768239554
Mean pinball loss with alpha = 0.9 prediction: 1562.0972423314806
Mean pinball loss with alpha = 0.9 MOD17: 2446.825938601763
Mean pinball loss with alpha = 0.09999999999999998 prediction: 1228.2488206067208
Mean pinball loss with alpha = 0.09999999999999998 MOD17: 946.820942608173


In [18]:
plot_result_distribution(downstream_test_df[TARGET_FEATURE], y_pred, modis_compare_data[TARGET_FEATURE], save_dir=f'{MODEL_RESULT_PLT_DIR}FT-Transformer_e2e.png', title='FT-Transformer(end-to-end) vs. MOD17')